In [2]:
from numpy import expand_dims, zeros, ones
from numpy.random import randn, randint
from keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Dense, Reshape, Flatten, Conv2D,\
Conv2DTranspose, LeakyReLU, Dropout, Embedding, Concatenate

In [3]:
def define_discriminator(in_shape=(28,28,1), n_classes=10):
    in_label = Input(shape=(1,))
    em = Embedding(n_classes, 50)(in_label)
    li = Dense(28*28)(em)
    li = Reshape((28,28,1))(li)
    in_image = Input(shape=in_shape)
    combined = Concatenate()([in_image, li])
    out = Conv2D(128, (3,3), strides=(2,2), padding='same')(combined)
    out = LeakyReLU(alpha=0.2)(out)
    out = Conv2D(128, (3,3), strides=(2,2), padding='same')(out)
    out = LeakyReLU(alpha=0.2)(out)
    out = Flatten()(out)
    out = Dropout(0.4)(out)
    out_layer = Dense(1, activation='sigmoid')(out)
    
    model = Model([in_image, in_label], out_layer)
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [4]:
def define_generator(latent_dim, n_classes=10):
    in_label = Input(shape=(1,))
    li = Embedding(n_classes, 50)(in_label)
    li = Dense(7*7)(li)
    li = Reshape((7,7,1))(li)
    input_dim = Input(shape=(latent_dim, ))
    gen = Dense(128*7*7)(input_dim)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((7,7,128))(gen)
    concat = Concatenate()([gen, li])
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(concat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
    
    model = Model([input_dim, in_label], out_layer)
    return model

In [5]:
def define_gan(g_model, d_model):
    d_model.trainable = False
    gen_noise, gen_label = g_model.input 
    gen_output = g_model.output
    gan_output = d_model([gen_output, gen_label])
    model = Model([gen_noise, gen_label], gan_output)
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [6]:
def load_real_samples():
    (trainX, trainy), (_, _) = load_data()
    X = expand_dims(trainX, axis=-1)
    X = X.astype('float32')
    X = (X - 127.5) / 127.5
    return [X, trainy]

In [7]:
def generate_real_samples(dataset, n_samples):
    images, labels = dataset
    idx = randint(0, images.shape[0], n_samples)
    X, labels = images[idx], labels[idx]
    y = ones((n_samples, 1))
    return [X, labels], y

In [8]:
def generate_latent_points(latent_dim, n_samples, n_classes=10):
    x_input = randn(latent_dim * n_samples)
    z_input = x_input.reshape(n_samples, latent_dim)
    labels = randint(0, n_classes, n_samples)
    return [z_input, labels]

In [9]:
def generate_fake_samples(generator, latent_dim, n_samples):
    z_input, labels = generate_latent_points(latent_dim, n_samples)
    images = generator.predict([z_input, labels])
    y = zeros((n_samples, 1))
    return [images, labels], y

In [10]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=300, n_batch=128):
    batch_per_epoch = int(dataset[0].shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    for i in range(n_epochs):
        for j in range(batch_per_epoch):
            [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
            d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
            [X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            d_loss2, _ = d_model.train_on_batch([X_fake, labels_fake], y_fake)
            
            [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
            y_gan = ones((n_batch, 1))
            
            g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
            
            print('> %d, %d/%d, real_loss=%.3f, fake_loss=%.3f, gan_loss=%.3f',
                  (i+1, j+1, batch_per_epoch, d_loss1, d_loss2, g_loss))
    g_model.save('cgan_generator.h5')
    d_model.save('cgan_discriminator.h5')


In [11]:
latent_dim = 100
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
dataset = load_real_samples()

In [12]:
train(g_model, d_model, gan_model, dataset, latent_dim)

In [17]:
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy 

In [20]:
def save_plot(examples, n):
    for i in range(n*n):
        plt.subplot(n, n, 1+i)
        plt.axis('off')
        plt.imshow(examples[i, :, :, 0], cmap='gray_r')
    plt.show()

In [21]:
model = load_model('cgan_generator.h5')
latent_points, labels = generate_latent_points(100, 100)
labels = numpy.asarray([x for _ in range(10) for x in range(10)]) 
X = model.predict([latent_points, labels])
X = X + 1 / 2.0
save_plot(X, 10)